title 전처리 및 자카르트 유사도, recall

In [2]:
import pandas as pd
import re

# CSV 불러오기
t0 = pd.read_csv("t0.csv", encoding='cp949')
t1 = pd.read_csv("t1.csv", encoding='utf-8')

# 1️⃣ title 전처리
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()  # 소문자 통일
    title = re.sub(r"[\-;,/():]", " ", title)  # 특수문자 제거
    title = re.sub(r"\s+", " ", title)  # 공백 통일
    title = title.strip()
    return title

t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 2️⃣ Jaccard 유사도 함수
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

threshold = 0.7  # 유사도 기준 (조정 가능)

# 3️⃣ title 유사도 기반 매칭
matches = []
for i, row0 in t0.iterrows():
    for j, row1 in t1.iterrows():
        sim = jaccard_similarity(row0["title"], row1["title"])
        if sim >= threshold:
            matches.append({
                "idx_t0": i,
                "idx_t1": j,
                "title_t0": row0["title"],
                "title_t1": row1["title"],
                "similarity": sim,
                "drug_t0": row0["drug"],
                "drug_t1": row1["drug"]
            })

matched_df = pd.DataFrame(matches)

# 4️⃣ drug 전처리 및 공통 여부 확인
def normalize_drug(drug_str):
    if pd.isna(drug_str):
        return []
    return [d.strip().lower() for d in drug_str.split(";")]

matched_df["drug_t0_list"] = matched_df["drug_t0"].apply(normalize_drug)
matched_df["drug_t1_list"] = matched_df["drug_t1"].apply(normalize_drug)
matched_df["drug_overlap"] = matched_df.apply(
    lambda x: len(set(x["drug_t0_list"]) & set(x["drug_t1_list"])) > 0, axis=1
)

# 5️⃣ 결과 확인
print(f"매칭된 title 개수: {matched_df.shape[0]}")
print(matched_df[["title_t0", "title_t1", "similarity", "drug_t0", "drug_t1", "drug_overlap"]].head(10))

# 6️⃣ drug 재현율 계산 (t0 기준)
recall = matched_df["drug_overlap"].sum() / t0["drug"].nunique()
print(f"Drug Recall (유사 title + drug 포함 기준): {recall:.4f}")


매칭된 title 개수: 29
                                            title_t0  \
0  conditionally reprogrammed colorectal cancer c...   
1  activation of the mtor pathway by oxaliplatin ...   
2  evaluation of the efficacy of dasatinib a src ...   
3  bgb 283 a novel raf kinase and egfr inhibitor ...   
4  synergistic anti tumor effect of mtor inhibito...   
5  integrative in vivo drug testing using gene ex...   
6  mtor inhibition and t dm1 in her2 positive bre...   
7  dual fatty acid synthase and her2 signaling bl...   
8  dz 2384 has a superior preclinical profile to ...   
9  jak2 regulates paclitaxel resistance in triple...   

                                            title_t1  similarity  \
0  conditionally reprogrammed colorectal cancer c...    1.000000   
1  activation of the mtor pathway by oxaliplatin ...    1.000000   
2  evaluation of the efficacy of dasatinib a src ...    0.842105   
3  bgb 283 a novel raf kinase and egfr inhibitor ...    0.842105   
4  synergistic anti tumor 

In [10]:
# 약물 소문자로 변환 후 ; 기준으로 리스트화
def normalize_drug(drug_str):
    return [d.strip().lower() for d in drug_str.split(";")]

matched_df["drug_t0_list"] = matched_df["drug_t0"].apply(normalize_drug)
matched_df["drug_t1_list"] = matched_df["drug_t1"].apply(normalize_drug)


In [11]:
# drug overlap 여부 확인
matched_df["drug_overlap"] = matched_df.apply(
    lambda x: len(set(x["drug_t0_list"]) & set(x["drug_t1_list"])) > 0, axis=1
)

# t0 기준 재현율
recall = matched_df["drug_overlap"].sum() / t0["drug"].nunique()
print(f"Drug Recall (유사 title + drug 포함 기준): {recall:.4f}")


Drug Recall (유사 title + drug 포함 기준): 0.1667


In [15]:
import pandas as pd
import re

# CSV 불러오기
# t0 = pd.read_csv("t0.csv")
# t1 = pd.read_csv("t1.csv")

# 1️⃣ title 전처리 함수
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()  # 소문자 통일
    title = re.sub(r"[\-;,/():]", " ", title)  # 특수문자 제거
    title = re.sub(r"\s+", " ", title)  # 공백 통일
    title = title.strip()
    return title

# 2️⃣ t0, t1의 title 컬럼 전처리
t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 3️⃣ 전처리 확인 (상위 5개)
print("=== t0 전처리된 title 예시 ===")
print(t0["title"].head())
print("\n=== t1 전처리된 title 예시 ===")
print(t1["title"].head())


=== t0 전처리된 title 예시 ===
0    conditionally reprogrammed colorectal cancer cells combined with mouse avatars identify synergy between egfr and mek or cdk4 6 inhibitors
1                  combining mek and src inhibitors for treatment of colorectal cancer demonstrate increased efficacy in vitro but not in vivo
2                                           sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6
3                                                                targeting ras mutant colorectal cancer with dual inhibition of mek and cdk4 6
4                                                mcl 1 inhibition overcomes intrinsic and acquired regorafenib resistance in colorectal cancer
Name: title, dtype: object

=== t1 전처리된 title 예시 ===
0                                                      cdk4 6 inhibitors impede chemoresistance and inhibit tumor growth of small cell lung cancer
1        conditionally reprogrammed colorectal cancer cells 

In [16]:
import pandas as pd
import re

# CSV 불러오기
# t0 = pd.read_csv("t0.csv")
# t1 = pd.read_csv("t1.csv")

# 1️⃣ title 전처리
def preprocess_title(title):
    if pd.isna(title):
        return ""
    title = title.lower()  # 소문자 통일
    title = re.sub(r"[\-;,/():]", " ", title)  # 특수문자 제거
    title = re.sub(r"\s+", " ", title)  # 공백 통일
    title = title.strip()
    return title

t0["title"] = t0["title"].apply(preprocess_title)
t1["title"] = t1["title"].apply(preprocess_title)

# 2️⃣ Jaccard 유사도 함수
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

threshold = 0.7  # 유사도 기준 (조정 가능)

# 3️⃣ title 유사도 기반 매칭
matches = []
for i, row0 in t0.iterrows():
    for j, row1 in t1.iterrows():
        sim = jaccard_similarity(row0["title"], row1["title"])
        if sim >= threshold:
            matches.append({
                "idx_t0": i,
                "idx_t1": j,
                "title_t0": row0["title"],
                "title_t1": row1["title"],
                "similarity": sim,
                "drug_t0": row0["drug"],
                "drug_t1": row1["drug"]
            })

matched_df = pd.DataFrame(matches)

# 4️⃣ drug 전처리 및 공통 여부 확인
def normalize_drug(drug_str):
    if pd.isna(drug_str):
        return []
    return [d.strip().lower() for d in drug_str.split(";")]

matched_df["drug_t0_list"] = matched_df["drug_t0"].apply(normalize_drug)
matched_df["drug_t1_list"] = matched_df["drug_t1"].apply(normalize_drug)
matched_df["drug_overlap"] = matched_df.apply(
    lambda x: len(set(x["drug_t0_list"]) & set(x["drug_t1_list"])) > 0, axis=1
)

# 5️⃣ 결과 확인
print(f"매칭된 title 개수: {matched_df.shape[0]}")
print(matched_df[["title_t0", "title_t1", "similarity", "drug_t0", "drug_t1", "drug_overlap"]].head(10))

# 6️⃣ drug 재현율 계산 (t0 기준)
recall = matched_df["drug_overlap"].sum() / t0["drug"].nunique()
print(f"Drug Recall (유사 title + drug 포함 기준): {recall:.4f}")


매칭된 title 개수: 29
  title_t0                                                                                                                                                                                \
0                                              conditionally reprogrammed colorectal cancer cells combined with mouse avatars identify synergy between egfr and mek or cdk4 6 inhibitors   
1                                                                                   activation of the mtor pathway by oxaliplatin in the treatment of colorectal cancer liver metastasis   
2                                                                    evaluation of the efficacy of dasatinib a src abl inhibitor in colorectal cancer cell lines and explant mouse model   
3                                                                    bgb 283 a novel raf kinase and egfr inhibitor displays potent antitumor activity in?braf mutated colorectal cancers   
4                                          

In [17]:
import pandas as pd

# 예시: t0, t1 이미 불러온 상태
# t0['drug'], t1['drug'] 컬럼 존재

# 1️⃣ drug 전처리 함수
def preprocess_drug(drug_str):
    if pd.isna(drug_str):
        return []
    # ; 구분, 소문자, 공백 제거
    return [d.strip().lower() for d in drug_str.split(";") if d.strip()]

# 2️⃣ t0, t1 drug 컬럼 전처리
t0["drug_list"] = t0["drug"].apply(preprocess_drug)
t1["drug_list"] = t1["drug"].apply(preprocess_drug)

# 3️⃣ 전처리 확인 (상위 5개)
print("=== t0 전처리된 drug ===")
print(t0[["drug", "drug_list"]].head())
print("\n=== t1 전처리된 drug ===")
print(t1[["drug", "drug_list"]].head())


=== t0 전처리된 drug ===
  drug                                            \
0  cetuximab; palbociclib; sorafenib; trametinib   
1                          dasatinib; trametinib   
2                        palbociclib; trametinib   
3                        palbociclib; trametinib   
4                            regorafenib; s63845   

  drug_list                                         
0  [cetuximab, palbociclib, sorafenib, trametinib]  
1                          [dasatinib, trametinib]  
2                        [palbociclib, trametinib]  
3                        [palbociclib, trametinib]  
4                            [regorafenib, s63845]  

=== t1 전처리된 drug ===
  drug                                                             \
0                                                   PALLADIUM;LEE   
1                                                               -   
2                               TRAMETINIB;NAVITOCLAX;NAVTEMADLIN   
3                                LIFIRAFENIB;DABRA

In [19]:
# ======================================
# 1️⃣ Drug 교집합 True/False 확인
# ======================================
def drug_overlap(row):
    # drug_t0와 drug_t1 리스트 간 하나라도 겹치면 True
    overlap = any(d in row["drug_t1"] for d in row["drug_t0"])
    print("====================================")
    print(f"Title T0: {row['title_t0']}")
    print(f"Title T1: {row['title_t1']}")
    print(f"Drug T0: {row['drug_t0']}")
    print(f"Drug T1: {row['drug_t1']}")
    print(f"Overlap: {overlap}")
    return overlap

matched_df["drug_overlap"] = matched_df.apply(drug_overlap, axis=1)

# ======================================
# 2️⃣ Drug Recall 계산
# ======================================
true_positives = matched_df["drug_overlap"].sum()
drug_recall = true_positives / len(matched_df) if len(matched_df) > 0 else 0

print("\n====================================")
print(f"Drug Recall (drug 교집합 1개 이상 기준): {drug_recall:.4f}")
print(f"Matched Titles: {len(matched_df)} / {len(t0)}")


Title T0: conditionally reprogrammed colorectal cancer cells combined with mouse avatars identify synergy between egfr and mek or cdk4 6 inhibitors
Title T1: conditionally reprogrammed colorectal cancer cells combined with mouse avatars identify synergy between egfr and mek or cdk4 6 inhibitors
Drug T0: ['CETUXIMAB', 'PALBOCICLIB', 'SORAFENIB', 'TRAMETINIB']
Drug T1: ['-']
Overlap: False
Title T0: activation of the mtor pathway by oxaliplatin in the treatment of colorectal cancer liver metastasis
Title T1: activation of the mtor pathway by oxaliplatin in the treatment of colorectal cancer liver metastasis
Drug T0: ['EVEROLIMUS', 'OXALIPLATIN']
Drug T1: ['OXALIPLATIN', 'EVEROLIMUS']
Overlap: True
Title T0: evaluation of the efficacy of dasatinib a src abl inhibitor in colorectal cancer cell lines and explant mouse model
Title T1: evaluation of the efficacy of dasatinib a src abl inhibitor in colorectal cancer cell lines and explant mousemodel
Drug T0: ['DASATINIB']
Drug T1: ['DASATINIB'

In [18]:
import pandas as pd
import re
from drugname_standardizer import DrugStandardizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ======================================
# 1️⃣ 전처리 함수
# ======================================
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_drug_list(drug_str):
    if pd.isna(drug_str):
        return []
    return [d.strip() for d in drug_str.split(";") if d.strip()]

# ======================================
# 2️⃣ CSV 불러오기 (예시)
# ======================================
# t0 = pd.read_csv("t0.csv", encoding="utf-8")
# t1 = pd.read_csv("t1.csv", encoding="utf-8")

# ======================================
# 3️⃣ 컬럼 전처리
# ======================================
for df_name, df in zip(["t0", "t1"], [t0, t1]):
    df["title"] = df["title"].apply(preprocess_text)
    df["drug_list"] = df["drug"].apply(preprocess_drug_list)
    print(f"=== {df_name} 전처리 확인 ===")
    print(df[["title", "drug_list"]].head(5))

# ======================================
# 4️⃣ Drug Generic 변환
# ======================================
ds = DrugStandardizer()

def convert_to_generic(drug_list):
    return [ds.standardize_name(d) for d in drug_list]

t0["drug_list"] = t0["drug_list"].apply(convert_to_generic)
t1["drug_list"] = t1["drug_list"].apply(convert_to_generic)

print("\n=== t0 drug_list generic 변환 확인 ===")
print(t0[["drug", "drug_list"]].head(5))
print("\n=== t1 drug_list generic 변환 확인 ===")
print(t1[["drug", "drug_list"]].head(5))

# ======================================
# 5️⃣ Title 매칭 (cosine similarity)
# ======================================
def match_titles(t0, t1, threshold=0.8):
    vec = TfidfVectorizer().fit(t0["title"].tolist() + t1["title"].tolist())
    t0_vec = vec.transform(t0["title"])
    t1_vec = vec.transform(t1["title"])
    
    matches = []
    for i, t0_row in t0.iterrows():
        sims = cosine_similarity(t0_vec[i], t1_vec).flatten()
        max_idx = sims.argmax()
        if sims[max_idx] >= threshold:
            matches.append({
                "title_t0": t0_row["title"],
                "title_t1": t1.iloc[max_idx]["title"],
                "drug_t0": t0_row["drug_list"],
                "drug_t1": t1.iloc[max_idx]["drug_list"],
                "similarity": sims[max_idx]
            })
    return pd.DataFrame(matches)

matched_df = match_titles(t0, t1, threshold=0.8)
print("\n=== Title 매칭 결과 확인 ===")
print(matched_df.head(10))

# ======================================
# 6️⃣ Drug 교집합 True/False
# ======================================
def drug_overlap(row):
    return any(d in row["drug_t1"] for d in row["drug_t0"])

matched_df["drug_overlap"] = matched_df.apply(drug_overlap, axis=1)
print("\n=== Drug Overlap True/False 확인 ===")
print(matched_df[["drug_t0", "drug_t1", "drug_overlap"]].head(10))

# ======================================
# 7️⃣ Drug Recall 계산
# ======================================
true_positives = matched_df["drug_overlap"].sum()
drug_recall = true_positives / len(matched_df) if len(matched_df) > 0 else 0
print(f"\nDrug Recall (drug 교집합 1개 이상 기준): {drug_recall:.4f}")
print(f"Matched Titles: {len(matched_df)} / {len(t0)}")


=== t0 전처리 확인 ===
  title                                                                                                                                       \
0  conditionally reprogrammed colorectal cancer cells combined with mouse avatars identify synergy between egfr and mek or cdk4 6 inhibitors   
1                combining mek and src inhibitors for treatment of colorectal cancer demonstrate increased efficacy in vitro but not in vivo   
2                                         sensitivity of kras mutant colorectal cancers to combination therapy that cotargets mek and cdk4 6   
3                                                              targeting ras mutant colorectal cancer with dual inhibition of mek and cdk4 6   
4                                              mcl 1 inhibition overcomes intrinsic and acquired regorafenib resistance in colorectal cancer   

  drug_list                                         
0  [cetuximab, palbociclib, sorafenib, trametinib]  
1          

In [20]:
# ======================================
# 6️⃣ Drug Recall (개별 행별 recall 포함)
# ======================================
def compute_row_recall(row):
    set_true = set(row["drug_t0"])
    set_pred = set(row["drug_t1"])
    if len(set_true) == 0:
        return None  # 정답이 없는 경우 제외
    inter = len(set_true & set_pred)
    recall = inter / len(set_true)
    return recall

# 각 행별 recall 계산
matched_df["row_recall"] = matched_df.apply(compute_row_recall, axis=1)

# 전체 평균 (nan 제외)
mean_row_recall = matched_df["row_recall"].dropna().mean()

# 기존 방식: True 비율 기반 recall
true_positives = matched_df["drug_overlap"].sum()
overall_recall = true_positives / len(matched_df) if len(matched_df) > 0 else 0

# 결과 출력
print("\n====================================")
print(f"🔹 행별 평균 재현율 (mean of per-row recall): {mean_row_recall:.4f}")
print(f"🔹 True 비율 기반 재현율 (overall recall): {overall_recall:.4f}")
print("====================================")

# 행별 상세 출력
for idx, row in matched_df.iterrows():
    print(f"Index: {idx}")
    print(f"Title T0: {row['title_t0'][:80]}...")
    print(f"Recall (row): {row['row_recall']}")
    print(f"Drug T0: {row['drug_t0']}")
    print(f"Drug T1: {row['drug_t1']}")
    print("------------------------------------")



🔹 행별 평균 재현율 (mean of per-row recall): 0.6293
🔹 True 비율 기반 재현율 (overall recall): 0.8276
Index: 0
Title T0: conditionally reprogrammed colorectal cancer cells combined with mouse avatars i...
Recall (row): 0.0
Drug T0: ['CETUXIMAB', 'PALBOCICLIB', 'SORAFENIB', 'TRAMETINIB']
Drug T1: ['-']
------------------------------------
Index: 1
Title T0: activation of the mtor pathway by oxaliplatin in the treatment of colorectal can...
Recall (row): 1.0
Drug T0: ['EVEROLIMUS', 'OXALIPLATIN']
Drug T1: ['OXALIPLATIN', 'EVEROLIMUS']
------------------------------------
Index: 2
Title T0: evaluation of the efficacy of dasatinib a src abl inhibitor in colorectal cancer...
Recall (row): 1.0
Drug T0: ['DASATINIB']
Drug T1: ['DASATINIB']
------------------------------------
Index: 3
Title T0: lowdose trametinib and bclxl antagonist have a specific antitumor effect in kras...
Recall (row): 0.5
Drug T0: ['abt263', 'TRAMETINIB']
Drug T1: ['TRAMETINIB', 'ABT263']
------------------------------------
Index: 4

In [25]:
# ======================================
# 6️⃣ Drug Recall (개별 행별 recall 포함)
# ======================================
def compute_row_recall(row):
    set_true = set(row["drug_t0"])
    set_pred = set(row["drug_t1"])
    if len(set_true) == 0:
        return None  # 정답이 없는 경우 제외
    inter = len(set_true & set_pred)
    recall = inter / len(set_true)
    return recall

def compute_row_precision(row):
    set_true = set(row["drug_t0"])
    set_pred = set(row["drug_t1"])
    if len(set_pred) == 0:
        return None  # 예측이 없는 경우 제외
    inter = len(set_true & set_pred)
    precision = inter / len(set_pred)
    return precision

# 행별 recall, precision, f1 계산
matched_df["row_recall"] = matched_df.apply(compute_row_recall, axis=1)
matched_df["row_precision"] = matched_df.apply(compute_row_precision, axis=1)

def compute_row_f1(row):
    p, r = row["row_precision"], row["row_recall"]
    if pd.isna(p) or pd.isna(r) or (p + r == 0):
        return None
    return 2 * p * r / (p + r)

matched_df["row_f1"] = matched_df.apply(compute_row_f1, axis=1)

# 전체 평균 (매크로)
macro_recall = matched_df["row_recall"].dropna().mean()
macro_precision = matched_df["row_precision"].dropna().mean()
macro_f1 = matched_df["row_f1"].dropna().mean()

# 마이크로 평균 계산 (전체 TP/FP/FN 합산 기준)
tp, fp, fn = 0, 0, 0
for _, row in matched_df.iterrows():
    set_true = set(row["drug_t0"])
    set_pred = set(row["drug_t1"])
    tp += len(set_true & set_pred)
    fp += len(set_pred - set_true)
    fn += len(set_true - set_pred)

micro_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
micro_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
micro_f1 = 2 * micro_precision * micro_recall / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

# 결과 출력
print("\n====================================")
print("🔹 행별 평균 지표 (매크로 평균)")
print(f"Macro Precision : {macro_precision:.4f}")
print(f"Macro Recall    : {macro_recall:.4f}")
print(f"Macro F1         : {macro_f1:.4f}")
print("------------------------------------")
print("🔸 전체 통합 지표 (마이크로 평균)")
print(f"Micro Precision : {micro_precision:.4f}")
print(f"Micro Recall    : {micro_recall:.4f}")
print(f"Micro F1         : {micro_f1:.4f}")
print("====================================")

# 행별 상세 출력 (원하면 주석 처리 가능)
for idx, row in matched_df.iterrows():
    print(f"Index: {idx}")
    print(f"Title T0: {row['title_t0'][:80]}...")
    print(f"Precision: {row['row_precision']}, Recall: {row['row_recall']}, F1: {row['row_f1']}")
    print(f"Drug T0: {row['drug_t0']}")
    print(f"Drug T1: {row['drug_t1']}")
    print("------------------------------------")



🔹 행별 평균 지표 (매크로 평균)
Macro Precision : 0.5530
Macro Recall    : 0.6293
Macro F1         : 0.6883
------------------------------------
🔸 전체 통합 지표 (마이크로 평균)
Micro Precision : 0.4800
Micro Recall    : 0.6234
Micro F1         : 0.5424
Index: 0
Title T0: conditionally reprogrammed colorectal cancer cells combined with mouse avatars i...
Precision: 0.0, Recall: 0.0, F1: nan
Drug T0: ['CETUXIMAB', 'PALBOCICLIB', 'SORAFENIB', 'TRAMETINIB']
Drug T1: ['-']
------------------------------------
Index: 1
Title T0: activation of the mtor pathway by oxaliplatin in the treatment of colorectal can...
Precision: 1.0, Recall: 1.0, F1: 1.0
Drug T0: ['EVEROLIMUS', 'OXALIPLATIN']
Drug T1: ['OXALIPLATIN', 'EVEROLIMUS']
------------------------------------
Index: 2
Title T0: evaluation of the efficacy of dasatinib a src abl inhibitor in colorectal cancer...
Precision: 1.0, Recall: 1.0, F1: 1.0
Drug T0: ['DASATINIB']
Drug T1: ['DASATINIB']
------------------------------------
Index: 3
Title T0: lowdose trameti

In [26]:
# ======================================
# ✅ 매칭 및 재현율 결과 저장 코드
# ======================================

# 1️⃣ 행별 매칭 결과 저장
output_path = "matched_result.csv"
matched_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# 2️⃣ 전체 통계 요약 딕셔너리 생성
summary_data = {
    "mean_row_recall": [mean_row_recall],
    "overall_recall": [overall_recall],
    "macro_recall": [macro_recall],
    "micro_recall": [micro_recall],
    "f1_score": [f1_score]
}

# 3️⃣ 요약정보를 새로운 시트(또는 아래줄)에 추가 저장
summary_df = pd.DataFrame(summary_data)

# 4️⃣ 기존 파일에 이어붙이기
with open(output_path, "a", encoding="utf-8-sig") as f:
    f.write("\n\n=== Summary Statistics ===\n")
summary_df.to_csv(output_path, mode="a", index=False, encoding="utf-8-sig")

print(f"✅ 매칭 및 재현율 결과가 '{output_path}'에 저장되었습니다.")


✅ 매칭 및 재현율 결과가 'matched_result.csv'에 저장되었습니다.
